In [156]:
import json
import pandas as pd
import numpy as np
import os

In [214]:
top_n_pred = [1,2,3]
models = ["load_m1", "load_m2", "load_m3"]
datasets = ["1_complete", "2_cf_cr_optional", "3_cp_cf_cr_optional", "4_complete_without_return_expressions",'5_short_dim']
n_repetitions = 3
shortners = ['','42_']
output_dir="../output/reports/json/"

In [215]:
def calculate_avg_dict(result_list_dict):
    return {
        'precision':sum(d['precision'] for d in result_list_dict) / len(result_list_dict),
        'recall':sum(d['recall'] for d in result_list_dict) / len(result_list_dict),
        'f1-score':sum(d['f1-score'] for d in result_list_dict) / len(result_list_dict),
        'support':sum(d['support'] for d in result_list_dict) / len(result_list_dict),
    }

In [254]:
results_unfiltered = dict()

index = pd.MultiIndex.from_product([models, datasets],
                                   names=['model', 'subtype'])

columns = pd.MultiIndex.from_product([top_n_pred, ['precision', 'recall', 'f1-score']],
                                     names=['top', 'stats'])

df_macro_avg_unfiltered = pd.DataFrame(index=index, columns=columns)
df_weighted_avg_unfiltered = pd.DataFrame(index=index, columns=columns)

for model in models:
    for dataset in datasets:
        for n_rep in range(n_repetitions):
            for top_n in top_n_pred: 
                for sh in shortners:
                    constructed_path = output_dir + model +"_"+ dataset +"_"+ str(n_rep) +"_"+ str(top_n) +"_"+ sh + "unfiltered" + ".json"
                    if not os.path.exists(constructed_path):
                        continue
                    with open(constructed_path , "r") as f:
                        json_file = json.load(f)
                        key_name = model +"_"+ dataset +"_"+ str(top_n) +"_"+ sh
                        if key_name in results_unfiltered:
                            results_unfiltered[key_name]["accuracy"].append(json_file["accuracy"])
                            results_unfiltered[key_name]["macro avg"].append(json_file["macro avg"])
                            results_unfiltered[key_name]["weighted avg"].append(json_file["weighted avg"])
                            if k == n_repetitions - 1:
                                results_unfiltered[key_name]["macro avg summary"] = calculate_avg_dict(results_unfiltered[key_name]["macro avg"])
                                results_unfiltered[key_name]["macro avg summary"]['accuracy'] = np.mean(results_unfiltered[key_name]["accuracy"])
                                results_unfiltered[key_name]["weighted avg summary"] = calculate_avg_dict(results_unfiltered[key_name]["weighted avg"])
                                results_unfiltered[key_name]["weighted avg summary"]['accuracy'] = np.mean(results_unfiltered[key_name]["accuracy"])
                                
                                if sh == '42_':
                                    df_macro_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'precision')] = results_unfiltered[key_name]["macro avg summary"]['precision']
                                    df_macro_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'recall')] = results_unfiltered[key_name]["macro avg summary"]['recall']
                                    df_macro_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'f1-score')] = results_unfiltered[key_name]["macro avg summary"]['f1-score']
                                    
                                    df_weighted_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'precision')] = results_unfiltered[key_name]["weighted avg summary"]['precision']
                                    df_weighted_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'recall')] = results_unfiltered[key_name]["weighted avg summary"]['recall']
                                    df_weighted_avg_unfiltered.at[(model,'5_short_dim'),(top_n, 'f1-score')] = results_unfiltered[key_name]["weighted avg summary"]['f1-score']
                                else:
                                
                                    #s = pd.Series(results[key_name]["macro avg summary"], name=key_name)
                                    df_macro_avg_unfiltered.at[(model,dataset),(top_n, 'precision')] = results_unfiltered[key_name]["macro avg summary"]['precision']
                                    df_macro_avg_unfiltered.at[(model,dataset),(top_n, 'recall')] = results_unfiltered[key_name]["macro avg summary"]['recall']
                                    df_macro_avg_unfiltered.at[(model,dataset),(top_n, 'f1-score')] = results_unfiltered[key_name]["macro avg summary"]['f1-score']

                                    #df_macro_avg = df_macro_avg.append(s)
                                    #s = pd.Series(results[key_name]["weighted avg summary"], name=key_name)
                                    #df_weighted_avg = df_weighted_avg.append(s)  

                                    df_weighted_avg_unfiltered.at[(model,dataset),(top_n, 'precision')] = results_unfiltered[key_name]["weighted avg summary"]['precision']
                                    df_weighted_avg_unfiltered.at[(model,dataset),(top_n, 'recall')] = results_unfiltered[key_name]["weighted avg summary"]['recall']
                                    df_weighted_avg_unfiltered.at[(model,dataset),(top_n, 'f1-score')] = results_unfiltered[key_name]["weighted avg summary"]['f1-score']
                        else:
                            results_unfiltered[key_name] = {
                                "accuracy":[json_file["accuracy"]],
                                "macro avg":[json_file["macro avg"]],
                                "weighted avg":[json_file["weighted avg"]]
                            }

In [255]:
results = dict()
df_macro_avg = pd.DataFrame(index=index, columns=columns)
df_weighted_avg = pd.DataFrame(index=index, columns=columns)

for model in models:
    for dataset in datasets:
        for n_rep in range(n_repetitions):
            for top_n in top_n_pred: 
                for sh in shortners: 
                    constructed_path = output_dir + model +"_"+ dataset +"_"+ str(n_rep) +"_"+ str(top_n)
                    if not sh == '':
                        constructed_path = constructed_path + "_" + sh[:-1]
                    constructed_path = constructed_path + ".json"
                    if not os.path.exists(constructed_path):
                        continue
                    with open(constructed_path , "r") as f:
                        json_file = json.load(f)
                        key_name = model +"_"+ dataset +"_"+ str(top_n) +"_"+ sh
                        if key_name in results:
                            results[key_name]["accuracy"].append(json_file["accuracy"])
                            results[key_name]["macro avg"].append(json_file["macro avg"])
                            results[key_name]["weighted avg"].append(json_file["weighted avg"])
                            if k == n_repetitions - 1:
                                results[key_name]["macro avg summary"] = calculate_avg_dict(results[key_name]["macro avg"])
                                results[key_name]["macro avg summary"]['accuracy'] = np.mean(results[key_name]["accuracy"])
                                results[key_name]["weighted avg summary"] = calculate_avg_dict(results[key_name]["weighted avg"])
                                results[key_name]["weighted avg summary"]['accuracy'] = np.mean(results[key_name]["accuracy"])
                                
                                if sh == '42_':
                                    df_macro_avg.at[(model,'5_short_dim'),(top_n, 'precision')] = results[key_name]["macro avg summary"]['precision']
                                    df_macro_avg.at[(model,'5_short_dim'),(top_n, 'recall')] = results[key_name]["macro avg summary"]['recall']
                                    df_macro_avg.at[(model,'5_short_dim'),(top_n, 'f1-score')] = results[key_name]["macro avg summary"]['f1-score']
                                    
                                    df_weighted_avg.at[(model,'5_short_dim'),(top_n, 'precision')] = results[key_name]["weighted avg summary"]['precision']
                                    df_weighted_avg.at[(model,'5_short_dim'),(top_n, 'recall')] = results[key_name]["weighted avg summary"]['recall']
                                    df_weighted_avg.at[(model,'5_short_dim'),(top_n, 'f1-score')] = results[key_name]["weighted avg summary"]['f1-score']
                                else:
                                
                                    #s = pd.Series(results[key_name]["macro avg summary"], name=key_name)
                                    df_macro_avg.at[(model,dataset),(top_n, 'precision')] = results[key_name]["macro avg summary"]['precision']
                                    df_macro_avg.at[(model,dataset),(top_n, 'recall')] = results[key_name]["macro avg summary"]['recall']
                                    df_macro_avg.at[(model,dataset),(top_n, 'f1-score')] = results[key_name]["macro avg summary"]['f1-score']

                                    #df_macro_avg = df_macro_avg.append(s)
                                    #s = pd.Series(results[key_name]["weighted avg summary"], name=key_name)
                                    #df_weighted_avg = df_weighted_avg.append(s)  

                                    df_weighted_avg.at[(model,dataset),(top_n, 'precision')] = results[key_name]["weighted avg summary"]['precision']
                                    df_weighted_avg.at[(model,dataset),(top_n, 'recall')] = results[key_name]["weighted avg summary"]['recall']
                                    df_weighted_avg.at[(model,dataset),(top_n, 'f1-score')] = results[key_name]["weighted avg summary"]['f1-score']

                                
                                    
                                    
                        else:
                            results[key_name] = {
                                "accuracy":[json_file["accuracy"]],
                                "macro avg":[json_file["macro avg"]],
                                "weighted avg":[json_file["weighted avg"]]
                            }

In [256]:
index
df = pd.DataFrame(index=index, columns=columns)
df.at[('load_m1', '1_complete'), (1,'precision')] = 5
df

top                                                   1                  \
stats                                         precision recall f1-score   
model   subtype                                                           
load_m1 1_complete                                    5    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   
load_m2 1_complete                                  NaN    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   
load_m3 1_complete                                  NaN    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   

top                                                   2                  \
stats                                         precision recall f1-score   
model   subtype                                                           
load_m1 1_complete                                  NaN    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   
load_m2 1_complete                                  NaN    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   
load_m3 1_complete                                  NaN    NaN      NaN   
        2_cf_cr_optional                            NaN    NaN      NaN   
        3_cp_cf_cr_optional                         NaN    NaN      NaN   
        4_complete_without_return_expressions       NaN    NaN      NaN   
        5_short_dim                                 NaN    NaN      NaN   

top                                                   3                  
stats                                         precision recall f1-score  
model   subtype                                                          
load_m1 1_complete                                  NaN    NaN      NaN  
        2_cf_cr_optional                            NaN    NaN      NaN  
        3_cp_cf_cr_optional                         NaN    NaN      NaN  
        4_complete_without_return_expressions       NaN    NaN      NaN  
        5_short_dim                                 NaN    NaN      NaN  
load_m2 1_complete                                  NaN    NaN      NaN  
        2_cf_cr_optional                            NaN    NaN      NaN  
        3_cp_cf_cr_optional                         NaN    NaN      NaN  
        4_complete_without_return_expressions       NaN    NaN      NaN  
        5_short_dim                                 NaN    NaN      NaN  
load_m3 1_complete                                  NaN    NaN      NaN  
        2_cf_cr_optional                            NaN    NaN      NaN  
        3_cp_cf_cr_optional                         NaN    NaN      NaN  
        4_complete_without_return_expressions       NaN    NaN      NaN  
        5_short_dim                     

# Results:


In [257]:
df_macro_avg

top                                                    1             \
stats                                          precision     recall   
model   subtype                                                       
load_m1 1_complete                              0.298325   0.304049   
        2_cf_cr_optional                        0.206436   0.218709   
        3_cp_cf_cr_optional                     0.199665   0.179353   
        4_complete_without_return_expressions   0.292487   0.302825   
        5_short_dim                             0.369757   0.365965   
load_m2 1_complete                             0.0634046  0.0647274   
        2_cf_cr_optional                       0.0259231  0.0294871   
        3_cp_cf_cr_optional                    0.0321887  0.0300253   
        4_complete_without_return_expressions  0.0641273  0.0641166   
        5_short_dim                            0.0654012  0.0680129   
load_m3 1_complete                              0.665508   0.636584   
        2_cf_cr_optional                        0.455983    0.37162   
        3_cp_cf_cr_optional                     0.538562   0.386729   
        4_complete_without_return_expressions   0.607685   0.587351   
        5_short_dim                             0.666396     0.6371   

top                                                               2            \
stats                                           f1-score  precision    recall   
model   subtype                                                                 
load_m1 1_complete                              0.283497   0.426856  0.401088   
        2_cf_cr_optional                        0.190675   0.351278  0.297806   
        3_cp_cf_cr_optional                     0.164904   0.346533  0.256095   
        4_complete_without_return_expressions   0.282042   0.431964  0.404218   
        5_short_dim                             0.349528    0.50242  0.463643   
load_m2 1_complete                             0.0590848   0.120167  0.106565   
        2_cf_cr_optional                       0.0249323  0.0633461  0.051867   
        3_cp_cf_cr_optional                    0.0263888  0.0734114  0.050009   
        4_complete_without_return_expressions  0.0599263   0.124432  0.108577   
        5_short_dim                            0.0625439   0.118615  0.105843   
load_m3 1_complete                              0.624684   0.787486  0.726508   
        2_cf_cr_optional                        0.354531   0.616131  0.459002   
        3_cp_cf_cr_optional                     0.385111   0.707583  0.477444   
        4_complete_without_return_expressions   0.572042   0.748478  0.692336   
        5_short_dim                              0.62687   0.792377  0.727416   

top                                                              3             \
stats                                           f1-score precision     recall   
model   subtype                                                                 
load_m1 1_complete                              0.392162  0.508988    0.46146   
        2_cf_cr_optional                        0.291623  0.463384   0.368655   
        3_cp_cf_cr_optional                     0.260822  0.459487   0.323421   
        4_complete_without_return_expressions   0.397569  0.525753   0.472711   
        5_short_dim                              0.46101  0.592111   0.530696   
load_m2 1_complete                              0.104615   0.16617   0.139616   
        2_cf_cr_optional                       0.0499338  0.107108  0.0812682   
        3_cp_cf_cr_optional                    0.0498451  0.118381  0.0780058   
        4_complete_without_return_expressions   0.107897   0.17395   0.147551   
        5_short_dim                             0.103365   0.16526   0.140668   
load_m3 1_complete                              0.732798  0.849399   0.774627   
        2_cf_cr_optional                        0.473352  0.712426   0.524332   
        3_cp_cf_cr_optional                    

In [258]:
df_macro_avg_unfiltered

top                                                    1             \
stats                                          precision     recall   
model   subtype                                                       
load_m1 1_complete                              0.363376   0.370728   
        2_cf_cr_optional                        0.255081   0.270585   
        3_cp_cf_cr_optional                     0.244076   0.238446   
        4_complete_without_return_expressions   0.360838   0.376262   
        5_short_dim                             0.445278   0.446478   
load_m2 1_complete                             0.0882148  0.0936073   
        2_cf_cr_optional                       0.0349141  0.0404502   
        3_cp_cf_cr_optional                    0.0428749  0.0437556   
        4_complete_without_return_expressions  0.0938285  0.0979165   
        5_short_dim                            0.0896884  0.0961263   
load_m3 1_complete                              0.738981   0.719662   
        2_cf_cr_optional                        0.537519   0.459222   
        3_cp_cf_cr_optional                     0.624858   0.509961   
        4_complete_without_return_expressions   0.681276   0.667116   
        5_short_dim                             0.730874   0.713787   

top                                                               2  \
stats                                           f1-score  precision   
model   subtype                                                       
load_m1 1_complete                              0.350149   0.468306   
        2_cf_cr_optional                        0.243533   0.352948   
        3_cp_cf_cr_optional                     0.220611    0.33168   
        4_complete_without_return_expressions   0.353496    0.47391   
        5_short_dim                             0.429634   0.547991   
load_m2 1_complete                             0.0845929   0.141775   
        2_cf_cr_optional                       0.0347274  0.0620884   
        3_cp_cf_cr_optional                     0.038194   0.072806   
        4_complete_without_return_expressions   0.090519   0.146862   
        5_short_dim                            0.0873215   0.140529   
load_m3 1_complete                              0.710487   0.822841   
        2_cf_cr_optional                        0.448279   0.640266   
        3_cp_cf_cr_optional                     0.510054   0.719305   
        4_complete_without_return_expressions   0.654736   0.783968   
        5_short_dim                             0.704922   0.819432   

top                                                                  \
stats                                             recall   f1-score   
model   subtype                                                       
load_m1 1_complete                              0.452636   0.441816   
        2_cf_cr_optional                        0.335045   0.319961   
        3_cp_cf_cr_optional                     0.296607   0.288876   
        4_complete_without_return_expressions   0.465032    0.45282   
        5_short_dim                             0.526111   0.519695   
load_m2 1_complete                              0.135742   0.129745   
        2_cf_cr_optional                        0.059946  0.0558838   
        3_cp_cf_cr_optional                    0.0610353  0.0581221   
        4_complete_without_return_expressions   0.136549   0.132659   
        5_short_dim                              0.13503   0.128896   
load_m3 1_complete                              0.789259   0.790084   
        2_cf_cr_optional                        0.535993   0.541377   
        3_cp_cf_cr_optional                     0.581081    0.59582   
        4_complete_without_return_expressions   0.750786    0.75004   
        5_short_dim                              0.78477   0.786442   

top                                                    3                        
stats                                          precision     recall   f1-score  
m

In [259]:
df_weighted_avg

top                                                   1                      \
stats                                         precision    recall  f1-score   
model   subtype                                                               
load_m1 1_complete                             0.509831  0.576738  0.518956   
        2_cf_cr_optional                        0.40429  0.477953  0.414712   
        3_cp_cf_cr_optional                    0.412791  0.481098  0.416135   
        4_complete_without_return_expressions  0.512073  0.587448  0.530281   
        5_short_dim                            0.543045  0.598827  0.546789   
load_m2 1_complete                              0.34003  0.452142  0.371898   
        2_cf_cr_optional                       0.251973  0.368031  0.286791   
        3_cp_cf_cr_optional                    0.270309  0.379709  0.300563   
        4_complete_without_return_expressions  0.344552  0.459544  0.378875   
        5_short_dim                            0.329481  0.448894  0.367135   
load_m3 1_complete                             0.724563   0.73177  0.711139   
        2_cf_cr_optional                       0.570757  0.572395  0.533562   
        3_cp_cf_cr_optional                     0.58891  0.582292  0.546196   
        4_complete_without_return_expressions  0.698279  0.715588  0.691269   
        5_short_dim                            0.718138  0.730589  0.711117   

top                                                   2                      \
stats                                         precision    recall  f1-score   
model   subtype                                                               
load_m1 1_complete                             0.659283  0.705016  0.655096   
        2_cf_cr_optional                        0.59639  0.621393   0.55792   
        3_cp_cf_cr_optional                    0.602254  0.621909  0.557194   
        4_complete_without_return_expressions   0.67538  0.718364  0.671094   
        5_short_dim                            0.692784   0.72738  0.683821   
load_m2 1_complete                             0.508102  0.590676  0.507526   
        2_cf_cr_optional                       0.412231  0.518098  0.416214   
        3_cp_cf_cr_optional                    0.430851  0.525777  0.425219   
        4_complete_without_return_expressions  0.511518  0.593649  0.510067   
        5_short_dim                            0.504766  0.588609   0.50386   
load_m3 1_complete                             0.834785   0.83729  0.821739   
        2_cf_cr_optional                       0.711624  0.708628  0.674401   
        3_cp_cf_cr_optional                    0.731511   0.71602  0.685038   
        4_complete_without_return_expressions  0.815443   0.82473  0.805987   
        5_short_dim                            0.832812  0.835322  0.820213   

top                                                   3                      
stats                                         precision    recall  f1-score  
model   subtype                                                              
load_m1 1_complete                             0.731536  0.772561  0.729611  
        2_cf_cr_optional                        0.68255  0.701594  0.646866  
        3_cp_cf_cr_optional                    0.687034  0.697756  0.640979  
        4_complete_without_return_expressions  0.748483  0.782129  0.741918  
        5_short_dim                            0.766093  0.793468  0.756833  
load_m2 1_complete                             0.583708  0.666194  0.590353  
        2_cf_cr_optional                       0.525155  0.600082  0.499896  
        3_cp_cf_cr_optional                    0.541318  0.605496  0.506533  
        4_complete_without_return_expressions  0.584843  0.668143  0.592486  
        5_short_dim                            0.579939  0.663497  0.587155  
load_m3 1_complete                              0.88211  0.882294  0.869586  
        2_cf_cr_optional                       0.782395  0.778

In [260]:
df_weighted_avg_unfiltered

top                                                   1                      \
stats                                         precision    recall  f1-score   
model   subtype                                                               
load_m1 1_complete                             0.603294  0.677066   0.61553   
        2_cf_cr_optional                       0.514376  0.620008  0.541554   
        3_cp_cf_cr_optional                    0.528633  0.628221  0.549218   
        4_complete_without_return_expressions  0.601511  0.685844  0.624567   
        5_short_dim                            0.638504  0.703196  0.647246   
load_m2 1_complete                              0.41198  0.546677  0.453644   
        2_cf_cr_optional                       0.343399  0.509506  0.399298   
        3_cp_cf_cr_optional                    0.365759  0.527126  0.418419   
        4_complete_without_return_expressions  0.422386   0.55922  0.466186   
        5_short_dim                            0.404073  0.546579  0.451428   
load_m3 1_complete                             0.817021  0.832035  0.811497   
        2_cf_cr_optional                       0.693211  0.699794  0.657737   
        3_cp_cf_cr_optional                    0.707457  0.711891    0.6739   
        4_complete_without_return_expressions  0.790905  0.812635  0.789048   
        5_short_dim                             0.81006  0.827516  0.808571   

top                                                   2                      \
stats                                         precision    recall  f1-score   
model   subtype                                                               
load_m1 1_complete                             0.717159  0.766585  0.718973   
        2_cf_cr_optional                       0.656498  0.696568  0.635333   
        3_cp_cf_cr_optional                    0.660666  0.700899  0.638538   
        4_complete_without_return_expressions  0.727702  0.774755  0.730195   
        5_short_dim                            0.747762  0.785699  0.744716   
load_m2 1_complete                             0.552172  0.638443   0.55529   
        2_cf_cr_optional                       0.442688  0.571592  0.465192   
        3_cp_cf_cr_optional                    0.456888  0.574111  0.469985   
        4_complete_without_return_expressions   0.55778  0.644942  0.562058   
        5_short_dim                            0.546671  0.637518  0.553479   
load_m3 1_complete                             0.883704  0.891727  0.878506   
        2_cf_cr_optional                       0.784658  0.782821  0.755715   
        3_cp_cf_cr_optional                    0.797593  0.792205  0.768311   
        4_complete_without_return_expressions  0.866307  0.880093  0.864011   
        5_short_dim                            0.880569   0.89015  0.877243   

top                                                   3                      
stats                                         precision    recall  f1-score  
model   subtype                                                              
load_m1 1_complete                             0.768418  0.812453  0.773022  
        2_cf_cr_optional                       0.726638  0.756533  0.708718  
        3_cp_cf_cr_optional                    0.729096  0.755517  0.705464  
        4_complete_without_return_expressions  0.781923  0.820217  0.783873  
        5_short_dim                            0.800531  0.832345  0.799612  
load_m2 1_complete                             0.616927  0.700047  0.627548  
        2_cf_cr_optional                       0.544344   0.63556  0.537012  
        3_cp_cf_cr_optional                    0.565156  0.640976  0.543573  
        4_complete_without_return_expressions  0.621892  0.705389  0.633951  
        5_short_dim                            0.610772  0.698266  0.625081  
load_m3 1_complete                             0.913661  0.918576  0.908634  
        2_cf_cr_optional                       0.834502  0.830

# Top Results:

## Precision: